<a href="https://colab.research.google.com/github/ShawnLiu119/Recommendation_ML_CB-CF-Hyb/blob/main/Recommendation_CB_CF_Hyb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Overview all regular recommendation system approach - CB/CF/Hybrid**

### **Term** <br>
**content based** should give you pretty safe recommendations. However, they may bound to what you already know your customers like.

**collaborative filtering approach** recommends items liked by customers whose choice is similar to yours

**cold start** for that particular customer or that particular item, meaning that you could not educatedly predict someone without knowing even a thing of that person: a **knowledge-based approach** would do the job to find out something of the newcomers by you **sending out questionnaire** to collect their personal preference with their consent.

**Recommendation strategy is dependent on how much you know your customers**

data summary<br>

 It contains events that a **customer view / add-to-cart / purchase an item**[INTERACTION], and the category hierarchy for each of the items. The ~235000 items are categorized in ~1300 categories and these categories are further grouped into some ~400 parent categories.
 https://medium.com/analytics-vidhya/content-based-collaborative-and-their-hybrid-approaches-for-recommendation-making-to-e-commence-e2015830a04f

Reference:https://github.com/recommenders-team/recommenders

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path = '/content/drive/MyDrive/kaggle_data/retail_recom'

category_tree = pd.read_csv(path + '/category_tree.csv')
events = pd.read_csv(path + '/events.csv')
item_properties_1 = pd.read_csv(path + '/item_properties_part1.csv')
item_properties_2 = pd.read_csv(path + '/item_properties_part2.csv')

In [4]:
category_tree.head()

,categoryid,parentid
0,1016,213.0
1,809,169.0
2,570,9.0
3,1691,885.0
4,536,1691.0


In [5]:
events

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN
...,...,...,...,...,...
2756096,1438398785939,591435,view,261427,NaN
2756097,1438399813142,762376,view,115946,NaN
2756098,1438397820527,1251746,view,78144,NaN
2756099,1438398530703,1184451,view,283392,NaN


In [9]:
events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2756101 entries, 0 to 2756100
Data columns (total 5 columns):
 #   Column         Dtype  
---  ------         -----  
 0   timestamp      int64  
 1   visitorid      int64  
 2   event          object 
 3   itemid         int64  
 4   transactionid  float64
dtypes: float64(1), int64(3), object(1)
memory usage: 105.1+ MB


In [6]:
item_properties_1.head()

,timestamp,itemid,property,value
0,1435460400000,460429,categoryid,1338
1,1441508400000,206783,888,1116713 960601 n277.200
2,1439089200000,395014,400,n552.000 639502 n720.000 424566
3,1431226800000,59481,790,n15360.000
4,1431831600000,156781,917,828513
